In [3]:
import pyspark
import os
import pickle

sc = pyspark.SparkContext(appName="BigDataProject")

In [4]:
my_dir = "./BooksDataset"
readFromSavedFile = True

In [5]:
sc

<SparkContext master=local[*] appName=BigDataProject>

In [6]:
#Read Authors

if not(readFromSavedFile):
    authors = []

    for file in os.listdir(my_dir):
        d = os.path.join(my_dir, file)
        if os.path.isdir(d):
            res = file
            authors.append(res)
    print(authors)

    sc.parallelize(authors).saveAsTextFile("./RDD/AuthorsName")


In [7]:
#Read Books per Authors
if not(readFromSavedFile):
    books = [[] for _ in range(len(authors))]


    for i in range(len(authors)):
        mydiraut = my_dir+"/"+authors[i]
        for file in os.listdir(mydiraut):
            d = os.path.join(mydiraut, file)
            books[i].append(file)

    print(books)

    sc.parallelize(books).saveAsTextFile("./RDD/BooksName")

In [8]:
def wordCount(file_path):
    words = sc.textFile(file_path).filter(bool) \
                .flatMap(lambda line: line.split(" ")) \
                .map(lambda w: w.lower()) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .map(lambda x: (x[1], x[0])) \
                .sortByKey(False)
    result = words.collect()

    return result


if not(readFromSavedFile):
    allWordsForAnAuthors = []

    for i in range(2,len(authors)):
        for j in range(len(books[i])):
            input_filename = os.path.join(my_dir, authors[i])
            input_filename = os.path.join(input_filename, books[i][j])
            print(input_filename)
            wordsOfABook= wordCount(input_filename)
            
            allWordsForAnAuthors.append(wordsOfABook)


        allWordsForAnAuthors = sc.parallelize(allWordsForAnAuthors)
        allWordsForAnAuthors.flatMap(lambda x : (x[0], x[1])).reduceByKey(lambda v1, v2: v1+v2).sortByKey(False)
        allWordsForAnAuthors.saveAsTextFile('./RDD/'+authors[i])
        
        

        print(f"{authors[i]} {allWordsForAnAuthors.take(100)}")
        allWordsForAnAuthors = []



In [9]:
#Leggo gli rdd
listOfAuthors = sc.textFile("./RDD/AuthorsName/*")
listOfBooks = sc.textFile("./RDD/BooksName/*")

authors = listOfAuthors.collect()
n_authors = len(authors)
#wordsCountAuthors = [[] for _ in range(n_authors)]
wordsCountAuthors = []

for i in range(n_authors):
    print(authors[i])
    tmp = f"./RDD/Books/{authors[i]}/*"
    rdd_i = sc.textFile(tmp).collect()
    wordsCountAuthors.append(rdd_i)

#wordsCountAuthors = sc.textFile("./RDD/Books/*/*")

Arthur Conan Doyle
Charles Dickens
Daniel Defoe
Edith Wharton
Jane Austen
Joseph Conrad
Lewis Carroll
Louisa May Alcott
Voltaire
William Shakespeare


In [10]:
print(len(wordsCountAuthors))
for i in range(10):
    wordsCountAuthors[i] = sc.parallelize(wordsCountAuthors[i])

10


In [62]:
rdd_list = [[] for _ in range(10)]
for i in range(10):
    for w in wordsCountAuthors[i].collect():
        cleaned_string = w.replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace("'", '')

        word_list = cleaned_string.split(', ')

        tuple_list = [(word_list[i], word_list[i + 1]) for i in range(0, len(word_list), 2)]

        rdd_list[i].append(sc.parallelize(tuple_list))
        #print(type(tuple_list))


In [69]:
#[Authors][#Books]
rdd_list[0][0].take(100)


[('1839', 'the'),
 ('1181', 'and'),
 ('988', 'a'),
 ('909', 'to'),
 ('883', 'of'),
 ('674', 'i'),
 ('534', 'you'),
 ('532', 'in'),
 ('506', 'that'),
 ('460', 'it'),
 ('440', 'his'),
 ('435', 'is'),
 ('433', 'was'),
 ('398', 'he'),
 ('396', 'her'),
 ('330', 'with'),
 ('302', 'have'),
 ('291', 'as'),
 ('291', 'had'),
 ('276', 'she'),
 ('266', 'for'),
 ('232', 'be'),
 ('227', 'at'),
 ('226', 'but'),
 ('212', 'not'),
 ('205', 'my'),
 ('194', 'which'),
 ('173', 'we'),
 ('172', 'from'),
 ('165', 'all'),
 ('154', 'would'),
 ('153', 'they'),
 ('153', 'what'),
 ('152', 'upon'),
 ('149', 'said'),
 ('145', 'very'),
 ('137', '"i'),
 ('136', 'so'),
 ('135', 'are'),
 ('134', 'this'),
 ('132', 'your'),
 ('129', 'there'),
 ('129', 'were'),
 ('126', 'do'),
 ('121', 'will'),
 ('119', 'one'),
 ('119', 'on'),
 ('117', 'little'),
 ('113', 'their'),
 ('112', 'if'),
 ('107', 'up'),
 ('107', 'him'),
 ('99', 'who'),
 ('99', 'should'),
 ('98', 'when'),
 ('98', 'me'),
 ('97', 'out'),
 ('94', 'two'),
 ('92', 'mus

In [ ]:
#sc.stop()

NameError: name 'sc' is not defined

In [ ]:
#Possibile soluzione ?
prova = [[] for i in range(11)]

lista = list(range(10, -1,-1))

rddlist = sc.parallelize(lista)

for i in range(len(lista)):

    prova[i].append(rddlist.collect())


prova[0]
sc.parallelize(prova)

prova[0][0][0]

#print(type(rddlist.collect()))

10